# MTI ANALYSIS

## Data Loading and Preliminary Processing 

In [2]:
import pandas as pd
import unidecode

In [3]:
# load the 'MTI' sheet from the combined_data.xlsx file into a dataframe
df = pd.read_excel('combined_data.xlsx', sheet_name= 'MTI')

In [7]:
# Set the options to display all columns
pd.set_option('display.max_columns', None)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Normalize the datetime to remove the time component
df['M_Cierre'] = df['M_Cierre'].dt.normalize()

# Convert the datetime back to string format
df['M_Cierre'] = df['M_Cierre'].dt.strftime('%d/%m/%Y')

# Remove any leading or trailing whitespaces from the 'M_Cierre' column
df['M_Cierre'] = df['M_Cierre'].str.strip()

# Remove any leading or trailing whitespaces from the 'M_Publicacion' column
df['M_Publicacion'] = df['M_Publicacion'].str.strip()

# Convert 'M_Publicacion' column to datetime format
df['M_Publicacion'] = pd.to_datetime(df['M_Publicacion'], format='%d/%m/%Y')

# Convert the datetime back to string format
df['M_Publicacion'] = df['M_Publicacion'].dt.strftime('%d/%m/%Y')


In [74]:
# adjust the display options to show all columns
pd.set_option('display.max_columns', None)


In [ ]:
# print the original dataframe
print('Original dataframe:')
print(df)

# create a dictionary to map old column names to new column names
new_column_names = {'P_contract':'Procedimiento','M_Estado':'Estado_1','M_Publicacion':'Fecha_Publicacion','M_Cierre':'Fecha_Cierre','M_Ultima':'Ultima_Actualizacion',
                    'M_SIGAF': 'Codigo_SIGAF', 'M_Body2': 'Actividad_Economica', 'M_Body3': 'Descripcion_Contratación','D_EntidadId': 'Institucion',
                    'D_UnidadId': 'Unidad_Adquisiciones', 'D_ClasificaciónId':'Clasificación_Comercial','D_CategoriaId':'Categoria',
                    'D_FuenteDeFinanciamientoI':'Fuente_Financiamiento','D_NormaAplicableId':'Norma_Aplicable','D_TipoProcedimientoId':'Tipo_Procedimiento',
                    'D_ModalidadId':'Modalidad_Contratación','D_VinculacionId':'Vinculacion_PAC','D_Estado':'Estado_2'}

# use the rename() method to change the column names
df = df.rename(columns=new_column_names)


In [ ]:
# Convert the columns to datetime objects and extract the date component in one step
date_cols = ['Fecha_Publicacion', 'Fecha_Cierre', 'Ultima_Actualizacion']
df[date_cols] = df[date_cols].apply(pd.to_datetime, errors='coerce').apply(lambda x: x.dt.date)

# Rename the columns and place them in the original position
df.columns = df.columns.str.replace('_date', '')  # remove the '_date' suffix from column names
cols = df.columns.tolist()
for col in reversed(date_cols):
    cols.insert(cols.index(col) + 1, col)
    cols.remove(col)
df = df[cols]


In [5]:
# drop old date columns 

df= df.drop(columns=['PageNo','M_Datos','Unidad_Adquisiciones','Modalidad_Contratación','D_EstudiPBCId'])

In [6]:
# Define non-repeated columns
non_repeated_cols = ['Procedimiento', 'P_no', 'Estado_1', 'Codigo_SIGAF','Fecha_Publicacion', 'Fecha_Cierre',
                     'Ultima_Actualizacion', 'M_Body1', 'Actividad_Economica', 'Descripcion_Contratación',
                     'Institucion', 'D_ClasificacionId', 'Categoria', 'Fuente_Financiamiento','Norma_Aplicable', 
                     'Tipo_Procedimiento','Vinculacion_PAC', 'Estado_2']

# Group by non-repeated columns
groups = df.groupby(non_repeated_cols, dropna=False)


In [13]:
# Extract values from repeated columns for each group and assign to new rows
new_rows = []
for name, group in groups:
    # Check if any column in the sequence has data
    has_data = False
    for i in range(9):
        if not group[f'D_ADJ_Monto_{i}'].isna().all():
            has_data = True
            break
    
    # Add new row if any column in the sequence has data
    if has_data:
        for i in range(9):
            proveedor = group[f'D_ADJ_ProveedorNombre_{i}'].iloc[0]
            ruccolv = group[f'D_ADJ_ProveedorRUCColV_{i}'].iloc[0]
            monto = group[f'D_ADJ_Monto_{i}'].iloc[0]
            if pd.isna(monto):
                continue
            renglones = group[f'D_ADJ_RenglonesColV_{i}'].iloc[0]
            beneficiario = group[f'D_ADJ_BeneficiarioFinal_{i}'].iloc[0]
            new_row = {col: group[col].iloc[0] for col in non_repeated_cols}
            new_row['Proveedor'] = proveedor
            new_row['RUCColV'] = ruccolv
            new_row['Monto'] = monto
            new_row['RenglonesColV'] = renglones
            new_row['BeneficiarioFinal'] = beneficiario
            new_rows.append(new_row)

# Convert list of new rows to dataframe
new_df = pd.DataFrame(new_rows)


In [ ]:
new_df.head(60)

In [15]:
new_df.to_excel('mti_data_to_cluster.xlsx', index=False)

In [ ]:
new_df.columns

###  Dolar to Cordoba conversion and currency sign removal

In [96]:
#Load the structured data 
df = pd.read_excel('/Users/ludwingmoncadabellorin/Downloads/nic_ds/mti_data_to_cluster.xlsx', sheet_name= 'Sheet1')

In [97]:
df.shape

(6530, 23)

#### In order to convert from Dolars to Cordobas we needed to get official exchange conversion from those years. We built a simple table with two columns: year and amount. The amount was the yearly mean of the exchange value cordobas per dolar. The data was extracted from the Banco Central de Nicaragua website.¶

In [4]:
data= pd.read_excel('/Users/ludwingmoncadabellorin/Downloads/nic_ds/USD_COR BCN.xlsx')

In [5]:
# Create Python dictionary where the keys are the 'year' column of a pandas DataFrame named 'data' and the values are the 'rate' column of the same DataFrame
exchange_rates = dict(zip(data['year'], data['rate']))

#### Once we had the data, we created a function that takes in a monetary value, a currency symbol, and a year, and returns the converted value in Nicaraguan cordobas.


In [ ]:
#Double Check US$ and C$ are the only currency symbols
currencies = df['Monto'].str[:2].unique()
currencies

In [7]:
# Split the Monto column into currency and amount columns
df[['moneda', 'monto']] = df['Monto'].str.split(n=1, expand=True)

In [8]:
# Delete commas from values in 'monto'
df['monto'] = pd.to_numeric(df['monto'].str.replace(',', ''), errors='coerce')

In [9]:
# Convert 'Monto' column to numeric type
df['monto'] = pd.to_numeric(df['monto'], errors='coerce')

In [10]:
def convert_currency(value, currency, year, exchange_rates):
    """
    Convert a monetary value from the original currency to C$ using the exchange rate
    for the given year.

    Args:
        value (float or int): The monetary value to convert.
        currency (str): The currency symbol for the original currency.
        year (int): The year of the exchange rate to use for the conversion.
        exchange_rates (dict): A dictionary containing exchange rates for each year.

    Returns:
        float: The converted monetary value in C$, or the original value if the currency
               is already in C$.

    Raises:
        ValueError: If the value is not a float or integer, the currency symbol is not
                    recognized, or there is no exchange rate available for the given year.
    """
    if not isinstance(value, (int, float)):
        raise ValueError(f"Invalid value: {value}")
    elif currency == 'C$':
        return value
    elif currency == 'US$':
        exchange_rate = exchange_rates.get(year, None)
        if exchange_rate:
            return value * exchange_rate
        else:
            raise ValueError(f"No exchange rate found for year {year}")
    else:
        raise ValueError(f"Invalid currency symbol: {currency}")


In [11]:
# Apply currency conversion to the Amount column
df['monto'] = df.apply(lambda row: convert_currency(row['monto'], row['moneda'], row['Fecha_Publicacion'].year, exchange_rates), axis=1)


In [ ]:
df.head()

In [50]:
df.to_csv('mti_data.csv', index=False)

In [13]:
# Drop moneda, since we do not need it anymore 
df = df.drop('moneda', axis=1)

In [14]:
#Let's rename monto to converted_monto
df = df.rename(columns={'monto': 'converted_monto'})

### There are 47 unique procedimientos. Most of them are redundant and can fit into known categories. We reviewed the law and tried to map them into known categories as closer to the law as possible. 

In [ ]:
unique_procedimientos = df['Procedimiento'].unique()
print(unique_procedimientos)


In [25]:
import unidecode

def rename_procedure(value):
    """
    Rename the procedure value based on the given rules.
    
    Args:
        value (str): The procedure value to be renamed.
    
    Returns:
        str: The renamed procedure value.
    """
    licitacion_selectiva = [
        "licitacion restringida",
        "licitacion selectiva",
    ]

    licitacion_publica = [
        "licitacion publica nacional",
        "licitacion publica internacional",
        "licitacion publica",
        "licitacion o concurso publico internacional limitado",
        "licitacion o concurso publico internacional",
        "licitacion o concurso publico internacional con oferta financiera",
        "licitacion o concurso publico nacional",
        "licitacion publica nacional obras",
        "licitacion publica nacional bienes",
        "licitacion publica internacional obras",
        "licitacion publica internacional bienes y servicios",
        "licitacion o concurso publico internacional bcie",
    ]

    contratacion_menor = [
        "compra por cotizacion",
        "compra por cotizacion menor",
    ]

    comparacion_de_precios = [
        "comparacion de precios",
        "comparacion de precios especial bienes / bid",
        "comparacion de precios especial obras / bid",
        "comparacion de precios obras",
        "comparacion de precios bienes y servicios",
    ]

    seleccion_por_criterio = [
        "seleccion basada en la calidad y costo",
        "seleccion basada en calidad y costo"
        "seleccion basada en las calificaciones del consultor",
        "seleccion basada en el menor costo",
        "seleccion con base en una sola fuente",
        "seleccion de firma consultora bid",
        "seleccion de firma consultora bm",
    ]

    por_concurso = [
        "concurso",
        "concurso publico internacional",
        "concurso firma consultora",
        "concurso publico nacional - bcie mtx",
    ]

    value = unidecode.unidecode(value.lower()) # convert to lowercase and remove accents
    
    if value in licitacion_selectiva:
        return "Licitacion Selectiva"
    elif value in licitacion_publica:
        return "Licitacion Publica"
    elif value in contratacion_menor:
        return "Contratacion Menor"
    elif value in comparacion_de_precios:
        return "Comparacion de Precios"
    elif value in seleccion_por_criterio:
        return "Seleccion por Criterio"
    elif value in por_concurso:
        return "Por Concurso"
    else:
        return value


df['renamed_procedimiento'] = df['Procedimiento'].apply(rename_procedure)


In [ ]:
unique_procedimientos = df['renamed_procedimiento'].unique()
print(unique_procedimientos)

In [ ]:
df.columns